In [2]:
import os
import wandb
from utils import AverageMeter, accuracy
from torch.utils.tensorboard import SummaryWriter
import os
from tqdm import tqdm
import logging
from utils import accuracy, save_checkpoint

In [3]:
from generate_dataset import TouchFolderLabel, CalandraLabel
from model import MultiModalMoCo

In [4]:
import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import random_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset

In [6]:
def logging(epoch, idx, train_loader_len, loss, acc1, acc5, losses, top1, top5, pretrain, train=True):
    if pretrain:
        print('Epoch: [{0}][{1}/{2}]\t'
            'Loss {loss:.4f}'.format(
            epoch, idx, train_loader_len, loss=loss))
        wandb.log({"training loss": loss}, step=epoch * train_loader_len + idx)
    else:
        print('Epoch: [{0}][{1}/{2}]\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                'Acc@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                epoch, idx, train_loader_len, loss=losses, top1=top1))
        if train:
            wandb.log({"training loss": loss,
                        "training accuracy": acc1[0],
                        "training top5 accuracy": acc5[0]}, 
                        step=epoch) 
        else:
            wandb.log({"validation loss": loss,
                        "validation accuracy": acc1[0],
                        "validation top5 accuracy": acc5[0]}, 
                        step=epoch)

In [7]:
epochs = 100
log_every_n_epochs = 1
batch_size = 64
num_workers = 4
log_every_n_epochs = 50
data_folder = "/home/fotis/PycharmProjects/calandra_dataset/objects_split_object_wise/"
num_channels = 6
momentum = 0.99
temperature = 0.07
lr = 0.0001
weight_decay = 1e-6
nn_model = 'resenet18'
intra_dim = 128
inter_dim = 128
weight_inter_tv = 1
weight_inter_vt = 1
weight_intra_vision = 1
weight_intra_tactile = 1
pretrained_encoder = False

In [8]:
from data_aug.contrastive_learning_dataset import ContrastiveLearningDataset

dataset = ContrastiveLearningDataset(root_folder='data')
train_dataset = dataset.get_dataset('calandra_label', 2)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                                   num_workers=num_workers, pin_memory=True)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# load model
model = MultiModalMoCo(n_channels=num_channels, m=momentum, T=temperature, nn_model=nn_model, intra_dim=intra_dim, inter_dim=inter_dim, weight_inter_t=weight_inter_tv, weight_inter_vt=weight_inter_vt, weight_intra_vision=weight_intra_vision, weight_intra_tactile=weight_intra_tactile, pretrained_encoder=pretrained_encoder)

In [5]:
# define loss function (criterion) and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training with gpu: {device}.")
optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                       last_epoch=-1)
writer = SummaryWriter()
logging.basicConfig(filename=os.path.join(writer.log_dir, 'training.log'), level=logging.DEBUG)
criterion = torch.nn.CrossEntropyLoss().to(device)
# Set number of training epochs
logging.info(f"Start MViTaC training for {epochs} epochs.")
logging.info(f"Training with gpu: {device}.")
best_acc = 0
wandb.init(project="mvitac_pretrain")
for epoch in range(epochs):
    loss_epoch = 0
    pbar = tqdm(total=len(train_loader))
    for idx, values in pbar:
        x_vision_q, x_vision_k, x_tactile_q, x_tactile_k, label = values
        model.train()
        
        # send to device
        x_vision_q = x_vision_q.to(device)
        x_vision_k = x_vision_k.to(device)

        x_tactile_q = x_tactile_q.to(device)
        x_tactile_k = x_tactile_k.to(device)

        # Forward pass to get the loss
        loss, logits, labels = model(x_vision_q, x_vision_k, x_tactile_q, x_tactile_k, epoch, idx, len(train_loader))
        loss_epoch += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    avg_loss = loss_epoch / len(train_loader)
    if epoch % log_every_n_epochs == 0:
        top1, top5 = accuracy(logits, labels, topk=(1, 5))
        writer.add_scalar('loss', avg_loss, global_step=epoch)
        writer.add_scalar('acc/top1', top1[0], global_step=epoch)
        writer.add_scalar('acc/top5', top5[0], global_step=epoch)
        writer.add_scalar('learning_rate', scheduler.get_last_lr()[0], global_step=epoch)
        if top1[0] > best_acc:
            best_acc = top1[0]
            save_checkpoint({
                'epoch': epoch,
                'arch': 'resnet18',
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, is_best=True, filename=os.path.join(writer.log_dir, f'checkpoint_best.pth.tar'))
        # torch.save(state, f'models/calandra/model_{args.task}_{epoch}_{args.batch_size}_best_object_wise_05_t05.pth')
        wandb.save('models/calandra/model_{}_best_object_wise.pth'.format(epoch))
        
    # warmup for the first 10 epochs
    if epoch >= 10:
        scheduler.step()
    logging.debug(f"Epoch: {epoch}\tLoss: {loss}\tTop1 accuracy: {top1[0]}")

    logging.info("Training has finished.")
    # save model checkpoints
    checkpoint_name = 'checkpoint_{:04d}.pth.tar'.format(epochs)
    save_checkpoint({
        'epoch': epochs,
        'arch': 'resnet18',
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }, is_best=False, filename=os.path.join(writer.log_dir, checkpoint_name))
    logging.info(f"Model checkpoint and metadata has been saved at {writer.log_dir}.")

Training with gpu: cuda.


NameError: name 'model' is not defined